In [26]:
#convert jsonl files on json
import json

input_path = '../data/interim/tweets_cleaned_wagner_schwartz_with_stance.jsonl'
output_path = '../data/interim/tweets_cleaned_wagner_schwartz_classified.json'
n_loop = 1000
tasks = []

with open(input_path, 'r', encoding='utf-8') as infile:
    count = 0
    for line in infile:
        if count <= n_loop:
            obj = json.loads(line)

            if obj["stance"] != "":
                tasks.append({
                    "text": obj["text"],
                    "cleaned_text": obj["cleaned_text"],
                    "id": obj["id"],
                    "stance": obj["stance"]
                })
                count+= 1
    print(f"Number of objects: {count}")
    

with open(output_path, 'w', encoding='utf-8') as outfile:
    json.dump(tasks, outfile, ensure_ascii=False, indent=2)



Number of objects: 1001


In [4]:
#remove duplicated tweets of json files
import json

def remove_duplicates_keep_classified(input_file, output_file):
    with open(input_file, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError as e:
            print(f"❌ JSON is malformed: {e}")
            return

    seen = {}

    for idx, item in enumerate(data, 1):
        if not isinstance(item, dict) or "id" not in item:
            print(f"⚠️ Skipping malformed object at index {idx}")
            continue

        tid = item["id"]

        if tid not in seen:
            seen[tid] = item
        else:
            # prefer classified version (stance != "")
            if seen[tid].get("stance", "") == "" and item.get("stance", "") != "":
                seen[tid] = item
            # if both classified, keep the first one

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(list(seen.values()), f, ensure_ascii=False, indent=2)

    print(f"Removed duplicates. Kept {len(seen)} unique items.")

# Example usage
remove_duplicates_keep_classified(
    "tweets_for_labelstudio_monark.json",
    "tweets_for_labelstudio_monark_unique.json"
)


Removed duplicates. Kept 4795 unique items.


In [11]:
#count classified tweets
import json

def count_classified(input_file):
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    total = len(data)
    classified = sum(1 for item in data if item.get("stance", "").strip() != "")
    unclassified = total - classified

    print(f"Total tweets: {total}")
    print(f"Classified tweets: {classified}")
    print(f"Unclassified tweets: {unclassified}")

# Example usage
count_classified("tweets_for_labelstudio_monark_unique.json")


Total tweets: 4794
Classified tweets: 600
Unclassified tweets: 4194


In [12]:
import json

def select_classified(input_file, output_file):
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    # keep only tweets with stance filled
    classified = [item for item in data if item.get("stance", "").strip() != ""]

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(classified, f, ensure_ascii=False, indent=2)

    print(f"Extracted {len(classified)} classified tweets into '{output_file}'.")

# Example usage
select_classified(
    "tweets_for_labelstudio_monark_unique.json",
    "tweets_for_labelstudio_monark_classified.json"
)


Extracted 600 classified tweets into 'tweets_for_labelstudio_monark_classified.json'.


In [27]:
import json
import re
from typing import List

URL_RE = re.compile(r'https?://\S+', flags=re.IGNORECASE)
MENTION_RE = re.compile(r'@([A-Za-z0-9_]{1,15})')  # Twitter/X handle up to 15 chars
HASHTAG_RE = re.compile(r'#([\w]+)', flags=re.UNICODE)  # \w covers Unicode letters/digits/_

def unique_preserve_order(items: List[str]) -> List[str]:
    seen = set()
    out = []
    for it in items:
        if it.lower() not in seen:
            seen.add(it.lower())
            out.append(it)
    return out

def strip_extra_spaces(s: str) -> str:
    # collapse whitespace and trim
    return re.sub(r'\s+', ' ', s).strip()

def build_fields_from_text(raw_text: str):
    # 1) Extract mentions / hashtags (lowercased, de-duplicated, order-preserving)
    mentions = unique_preserve_order([m.lower() for m in MENTION_RE.findall(raw_text or "")])
    hashtags = unique_preserve_order([h.lower() for h in HASHTAG_RE.findall(raw_text or "")])

    # 2) Remove URLs
    no_urls = URL_RE.sub(' ', raw_text or '')

    # 3) Build TEXT FOR MODEL:
    #    remove @mentions but KEEP hashtags inline
    text_model = re.sub(r'@([A-Za-z0-9_]{1,15})', ' ', no_urls)
    text_model = strip_extra_spaces(text_model)

    # 4) Build a CLEAN TEXT (optional): remove mentions AND hashtags
    clean_text = HASHTAG_RE.sub(' ', re.sub(r'@([A-Za-z0-9_]{1,15})', ' ', no_urls))
    clean_text = strip_extra_spaces(clean_text)

    return text_model, clean_text, hashtags, mentions

def transform_dataset(input_file: str, output_file: str):
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    out = []
    for item in data:
        # prefer original "text" for parsing hashtags/mentions
        raw_text = item.get("text", "") or ""
        stance = item.get("stance", "")

        text_model, clean_text, hashtags, mentions = build_fields_from_text(raw_text)

        out.append({
            "id": item.get("id"),
            # text for the model: URLs + @mentions removed, hashtags KEPT inline
            "text": text_model,
            # optional extra clean version (no urls/@/hashtags) if you want to experiment
            "clean_text": clean_text,
            "hashtags": hashtags,    # e.g. ["monark", "flowpodcast"]
            "mentions": mentions,    # e.g. ["ratoborrachudo", "igor_3k"]
            "stance": stance
        })

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(out, f, ensure_ascii=False, indent=2)

    print(f"Transformed {len(out)} items → {output_file}")

# Example usage
transform_dataset("../data/interim/tweets_cleaned_wagner_schwartz_classified.json", "../data/interim/tweets_wagner_schwartz_final_for_bert.json")


Transformed 1001 items → ../data/interim/tweets_wagner_schwartz_final_for_bert.json


In [18]:
#prepare tweets into organized json files
from pathlib import Path
import json
from dateutil import parser
import re
import string
from unidecode import unidecode

def clean_texts(text):
    if not isinstance(text, str):
        return "", ""

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|pic\.twitter\.com\S+", "", text)
    
    # Remove mentions and hashtags (optional for transformers)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    
    # Remove multiple spaces
    text = re.sub(r"\s+", " ", text).strip()

    # === Version 1: for Transformers ===
    cleaned_text = text

    # === Version 2: for Classic ML (BoW, TF-IDF) ===
    text_bow = cleaned_text.lower()
    text_bow = unidecode(text_bow)  # Remove accents
    text_bow = re.sub(rf"[{re.escape(string.punctuation)}]", "", text_bow)  # Remove punctuation
    text_bow = re.sub(r"\s+", " ", text_bow).strip()

    return cleaned_text, text_bow


def extract_tweets_from_file(file_path):
    try:
        with open(file_path, 'r', encoding="utf-8") as file:
            data = json.load(file)
        tweets = []
        seen_ids = set()
        for item in data:
            instructions = item.get("data", {}).get("data", {}).get("search_by_raw_query", {}).get("search_timeline", {}).get("timeline", {}).get("instructions", [])
            if not instructions:
                continue
            entries = instructions[0].get("entries", [])
            for entry in entries:
                if not entry.get("entryId", "").startswith("tweet-"):
                    continue
                tweet = entry.get("content", {}).get("itemContent", {}).get("tweet_results", {}).get("result", {})
                legacy = tweet.get("legacy", {})
                tweet_id = tweet.get("rest_id")
                if not tweet_id or tweet_id in seen_ids:
                    continue
                full_text = legacy.get("full_text", "").strip()
                created_at = legacy.get("created_at", "")
                created_at_iso = ""
                if created_at:
                    try:
                        created_at_iso = parser.parse(created_at).isoformat()
                    except:
                        pass

                likes = legacy.get("favorite_count", "")
                is_retweet = "retweeted_status_result" in tweet
                is_quote = legacy.get("is_quote_status", "")
                in_reply_to = legacy.get("in_reply_to_screen_name", "")
                in_reply_to_status_id = legacy.get("in_reply_to_status_id_str", "")
                user = tweet.get("core", {}).get("user_results", {}).get("result", {}).get("core", {}).get("screen_name")

                user_mentions = legacy.get("entities", {}).get("user_mentions", [])
                mentions = []
                if user_mentions:
                    for mention in user_mentions:
                        mentions.append({
                            "id_str": mention.get("id_str", ""),
                            "name": mention.get("name", ""),
                            "username": mention.get("screen_name", ""),
                        })

                if full_text:
                    cleaned, cleaned_bow = clean_texts(full_text)
                    seen_ids.add(tweet_id)
                    tweets.append({
                        "id": tweet_id,
                        "user": user,
                        "text": full_text,
                        "cleaned_text" : cleaned,
                        "cleaned_text_bow" :cleaned_bow,
                        "created_at": created_at,
                        "created_at_iso" : created_at_iso,
                        "stance" : "",
                        "is_quote" : is_quote,
                        "is_retweet": is_retweet,
                        "tweet_type": "retweet" if is_retweet else ("quote" if is_quote else "original"),
                        "in_reply_to_user": in_reply_to,
                        "in_reply_to_status_id": in_reply_to_status_id,
                        "like_count": likes,
                        "hashtags" : [tag["text"] for tag in legacy.get("entities", {}).get("hashtags", [])],
                        "mentions" : mentions,
                        "retweet_count": legacy.get("retweet_count", 0),
                        "quote_count": legacy.get("quote_count", 0),
                        "reply_count": legacy.get("reply_count", 0)
                    })
        return tweets
    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")
        return []

all_tweets = []
raw_folder = Path("../data/raw/WagnerSchwartz")

for file_path in raw_folder.glob("*.json"):
    print(f"🔍 Processing: {file_path.name}")
    tweets = extract_tweets_from_file(file_path)
    all_tweets.extend(tweets)

# Write to .jsonl
output_path = Path("../data/interim/tweets_cleaned_wagner_schwartz.jsonl")
with output_path.open("w", encoding="utf-8") as out:
    for tweet in all_tweets:
        json.dump(tweet, out, ensure_ascii=False)
        out.write("\n")

print(f"✅ Done! {len(all_tweets)} tweets saved to {output_path}")


🔍 Processing: search_response_%22La+B%C3%AAte%22+%23ForaMAMBolsonaro2018_2017-01-01_2017-12-31_20250728_165718.json
🔍 Processing: search_response_%22La+b%C3%AAte%22_2017-01-01_2017-12-31_20250728_174921.json
🔍 Processing: search_response_%22pelad%C3%A3o+do+MAM%22_2017-01-01_2017-12-31_20250728_181133.json
🔍 Processing: search_response_%22Qual+vai+ser+a+desculpa+dessa+vez%3F%22+%28from%3Amblivre%29+lang%3Apt+until%3A2018-01-01+since%3A2017-09-01_2017-01-01_2017-12-31_20250731_163255.json
🔍 Processing: search_response_%22Wagner+Schwartz%22+%22ped%C3%B3filo%22_2017-01-01_2017-12-31_20250728_173815.json
🔍 Processing: search_response_%22Wagner+Schwartz%22+%28ped%C3%B3filo+OR+degenerado+OR+preso+OR+lixo+OR+doente%29_2017-01-01_2017-12-31_20250728_165425.json
🔍 Processing: search_response_%22Wagner+Schwartz%22+OR+%22La+B%C3%AAte%22+29_2017-01-01_2017-12-31_20250728_120552.json
🔍 Processing: search_response_%28%22liberdade+art%C3%ADstica%22+OR+%22censura%22+OR+%22arte+moderna2%29_2017-01-01_20

In [20]:
import json
from datetime import datetime

def parse_labelstudio_stances(ls_json_path):
    """
    Build a dict: tweet_id (string) -> stance (string) from a Label Studio
    export like the example you sent.
    If multiple annotations exist, we pick the one with the latest updated_at.
    """
    with open(ls_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    stance_by_id = {}

    for item in data:
        # tweet id is under item["data"]["id"]
        tweet_id = str(item.get("data", {}).get("id", "")).strip()
        if not tweet_id:
            continue

        annots = item.get("annotations", []) or []
        if not annots:
            continue

        # choose the latest annotation by updated_at (fallback to created_at)
        def _parse_dt(annot):
            ts = annot.get("updated_at") or annot.get("created_at")
            try:
                return datetime.fromisoformat(ts.replace("Z", "+00:00")) if ts else datetime.min
            except Exception:
                return datetime.min

        annots_sorted = sorted(annots, key=_parse_dt, reverse=True)
        chosen = annots_sorted[0]

        # extract stance from the first result->value->choices[0]
        stance = None
        for res in chosen.get("result", []) or []:
            val = res.get("value", {})
            choices = val.get("choices", [])
            if choices:
                stance = choices[0]
                break

        if stance is not None:
            stance_by_id[tweet_id] = stance

    return stance_by_id


def merge_stance_into_jsonl(ls_json_path, jsonl_in_path, jsonl_out_path):
    # build stance dict from Label Studio JSON
    id2stance = parse_labelstudio_stances(ls_json_path)

    total = 0
    updated = 0
    missing = 0

    with open(jsonl_in_path, "r", encoding="utf-8") as fin, \
         open(jsonl_out_path, "w", encoding="utf-8") as fout:

        for line in fin:
            if not line.strip():
                continue
            total += 1
            obj = json.loads(line)

            tid = str(obj.get("id", "")).strip()
            if tid and tid in id2stance:
                obj["stance"] = id2stance[tid]
                updated += 1
            else:
                # keep line as-is if no stance for this id
                missing += 1

            fout.write(json.dumps(obj, ensure_ascii=False) + "\n")

    print(f"Processed {total} lines.")
    print(f"Updated with stance: {updated}")
    print(f"No stance found for id: {missing}")


# Example usage
merge_stance_into_jsonl(
    "../data/processed/20250909_classified_tweets.json",
    "../data/interim/tweets_cleaned_wagner_schwartz.jsonl",
    "../data/interim/tweets_cleaned_wagner_schwartz_with_stance.jsonl"
)


Processed 4525 lines.
Updated with stance: 2026
No stance found for id: 2499
